We computed electronic couplings between the various states at QM level.
We can model the kinetics of the generation of the Charge Separated State with a set of coupled differential equations describing the population of each state.

\begin{align}
\dot{P}_{S_0} & = (k_{em} + k_{nr}) \cdot P_{S_1} \\
\dot{P}_{S_1} & =  -(k_{em} + k_{nr} + k_{ISC}) \cdot P_{S_1} + k_{rISC} P_{T_1} \\
\dot{P}_{T_1} & = k_{ISC} \cdot P_{S_1} - k_{rISC} \cdot P_{T_1} \\
\end{align}

Gathering populations of state in a vector $\mathbf{P}$ we can write this set of coupled equations in matrix form, as in

\begin{equation}
\dot{\mathbf{P}}(t) = \mathbf{K} \mathbf{P}(t)
\end{equation}

where we have defined the matrix of rate constants $\mathbf{K}$ as in the following.

\begin{equation}
\mathbf{K} = \begin{pmatrix} 0 & (k_{em} + k_{nr}) & 0 \\ 0 & -(k_{em} + k_{ISC} + k_{nr}) & k_{rISC} \\ 0 & k_{ISC} & -k_{rISC} \end{pmatrix}
\end{equation}

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint

df = pd.read_csv("rate_constants.csv")

mdls = df["Dye"]
rates = df[["k_em / Hz", "k_ISC / Hz", "k_rISC / Hz", "k_nr / Hz"]]

In [ ]:
# Define rate constants matrix
def make_K_matrix(*rates):
    
    K = np.zeros((3,3))
    K[0,1] = rates[0] + rates[-1] # (k_em + k_nr)
    K[1,1] = -(rates[0] + rates[1] + rates[-1]) # -(k_em + k_ISC + k_nr)
    K[1,2] = rates[2] # k_rISC
    K[2,1] = rates[1] # k_ISC
    K[2,2] = -rates[2] # -k_rISC
    
    return K

# Define population derivatives in time
def dPdt(P, t, K):
    return np.dot(K, P)

# Define time window
t = np.arange(0, 1e-2, 1e-10)

# Define initial conditions
P0 = np.zeros(3)

# Initial population from electrical excitation
P0[1] = 0.25
P0[2] = 0.75

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, ticker, gridspec
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)


# Define a function to plot solutions to the system of coupled ODEs for each
# model
def plot_pops(t, sol, mdl_name):
    
    fig = plt.figure(figsize=(12, 9))
    gs = gridspec.GridSpec(1, 1)

    ax = plt.subplot(gs[0])

    ax.plot(t, sol[:,0], lw=2, color="b", label=r"S$_0$")
    ax.plot(t, sol[:,1], lw=2, color="g", label=r"S$_1$")
    ax.plot(t, sol[:,2], lw=2, color="r", label=r"T$_1$")

    ax.set_xlabel(r"$t$ / s", size=30, labelpad=5)
    ax.set_ylabel(r"$P$", size=30, labelpad=5)

    # ax.set_xlim(0,1e6)
    ax.set_ylim(0, 1)
    ax.set_xscale('log')
    
    # xtickmaj = ticker.MultipleLocator(100)
    # xtickmin = ticker.AutoMinorLocator(5)
    ytickmaj = ticker.MultipleLocator(0.25)
    ytickmin = ticker.AutoMinorLocator(4)
    
    # ax.xaxis.set_major_locator(xtickmaj)
    # ax.xaxis.set_minor_locator(xtickmin)
    ax.yaxis.set_major_locator(ytickmaj)
    ax.yaxis.set_minor_locator(ytickmin)
    ax.xaxis.set_ticks_position('both')
    ax.yaxis.set_ticks_position('both')
    ax.tick_params(axis='both', which='major', direction='in', labelsize=28, pad=10, length=5)
    ax.tick_params(axis='both', which='minor', direction='in', labelsize=28, pad=10, length=2)

    ax.legend(loc=1, fontsize=22, frameon=False)

    plt.title("%s" % mdl_name, size=32)
    plt.savefig("%s.svg" % mdl_name.replace(" ", "_"), dpi=600)
    plt.show()
    
    return

In [ ]:
# Loop over models
for i, mdl in enumerate(mdls.values):
    
    # Define model name
    mdl_name = mdl
    
    # Get avg rates
    ks = rates.values[i] # * 1e-12
    
    # Define rate constants matrix
    K = make_K_matrix(*ks)
    
    # Numerically integrate the system of ODEs
    sol = odeint(dPdt, P0, t, args=(K,))
    
    # plot the solutions
    plot_pops(t, sol, mdl_name)